In [33]:
!pip install langchain openai transformers -q

# Getting started with Chains
Chains are the core of LangChain. They are simply a chain of components, executed in a particular order.

The simplest of these chains is the `LLMChain`. It works by taking a user's input, passing in to the first element in the chain — a `PromptTemplate` — to format the input into a particular prompt. The formatted prompt is then passed to the next (and final) element in the chain — a LLM.

We'll start by importing all the libraries that we'll be using in this example.

In [34]:
import re
import inspect
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain, SimpleSequentialChain
from langchain.callbacks import get_openai_callback

In [35]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [36]:
llm = OpenAI(temperature = 0)

In [37]:
# Function will tell how many tokens we are using in each call

def count_tokens(chain, query):
  with get_openai_callback() as cb:
    result = chain.run(query)
    print(f'Spent a total of {cb.total_tokens} tokens')
  return result

## Why chains?
In simple terms, a "chain" is like a series of steps in a process. These steps use different tools, like prompts or functions, to work on some input and give a result. There are three types of chains: **Utility chains, Generic chains, and Combine Documents chains**. We'll talk about Utility and Generic chains in this explanation.

1.   Utility Chains are like ready-made tools that quickly get specific information from a source.
2.   Generic Chains are like basic building blocks for making more complex tools, but you can't use them on their own.
</br>

Now, let's see what these chains can do!

# Utility Chains

1. LLMMathChain - Allows llms the ability to do maths.

In [38]:
llm_math_chain = LLMMathChain.from_llm(
    llm = llm,
    verbose = True # to see the steps llm is taking to complete the task
)

In [39]:
count_tokens(llm_math_chain, "What is the 23 to the power of 0.456?")



> Entering new LLMMathChain chain...
What is the 23 to the power of 0.456?
```text
23**0.456
```
...numexpr.evaluate("23**0.456")...

Answer: 4.17780238247165
> Finished chain.
Spent a total of 265 tokens


'Answer: 4.17780238247165'

In [40]:
count_tokens(llm_math_chain, "What will be percentage of marks if someone secured 439 out of 500?")



> Entering new LLMMathChain chain...
What will be percentage of marks if someone secured 439 out of 500?
```text
(439/500)*100
```
...numexpr.evaluate("(439/500)*100")...

Answer: 87.8
> Finished chain.
Spent a total of 270 tokens


'Answer: 87.8'

In [41]:
count_tokens(llm_math_chain, "A had 4 apples and 20 banana. The cost of a apple is twice of a banana. If the cost of whole apple is 200, then what is the cost of banana?")



> Entering new LLMMathChain chain...
A had 4 apples and 20 banana. The cost of a apple is twice of a banana. If the cost of whole apple is 200, then what is the cost of banana?
```text
200 / (4 * 2)
```
...numexpr.evaluate("200 / (4 * 2)")...

Answer: 25.0
> Finished chain.
Spent a total of 293 tokens


'Answer: 25.0'

Let's find out what's happening here. Someone asked a question using regular language, and the chain passed it on to the language model (llm). The llm then provided a Python code as the answer, which the chain used to give us the final answer. Now, you might wonder how the llm knew to give us Python code.

Well, it's because of something called prompts. The question we send to the chain isn't the only thing the llm looks at. The question is put into a larger context, like a set of instructions, which helps the llm understand how to deal with our question. This set of instructions is called a prompt. Now, let's find out what specific prompt this chain is using!

In [42]:
print(llm_math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



okay, let's break it down. We're basically telling the computer not to solve difficult math problems by itself. Instead, it should show us a piece of Python code that can solve the math problem. If we didn't give this instruction, the computer might attempt to solve the problem on its own, and it might not succeed. Let's test this and see what happens! 🧐

In [53]:
prompt = PromptTemplate(input_variables = ['question'], template = '{question}')
llm_chain = LLMChain(prompt = prompt, llm = llm)


count_tokens(llm_chain, "What is 54 raised to the power of 0.689?")

Spent a total of 18 tokens


'\n\n2.945'

Incorrect response! </br>
**Here's an important insight:** If we use prompts cleverly, we can make the language model (llm) avoid common mistakes by specifically and intentionally instructing it to act in a particular manner.

Another cool thing about this process is that it doesn't just input information into the llm; it also puts together Python code. Let's take a closer look at how this process functions.

In [60]:
print(inspect.getsource(llm_math_chain._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



Here, if the llm gives us a simple math answer, we're good. But if it gives Python code, we use a **Python REPL simulator** to run and get the exact answer for harder problems. Cool!
</br>
</br>
This is the chain process: llm gives an answer for easy math or Python code for tough problems, which we compile. Also, we see chain composition in action. We use `LLMMathChain`, which uses `LLMChain` when needed. We can make many such compositions, connecting chains for complex and customizable behavior.
</br>
</br>
Utility chains follow a similar structure: a prompt to make llm give a specific response type. We can ask for SQL queries, API calls, or Bash commands on the go 🔥
</br>
</br>
Langchain keeps growing, getting more flexible and powerful. Explore it and try the example notebooks.
</br>
</br>
**A Python REPL (Read-Eval-Print Loop) is an interactive shell for executing Python code line by line italicized text*

# Generic Chains

In langchain, there are just three main types of chains, and we'll focus on demonstrating all of them in one example. Ready?
</br></br>
Imagine dealing with messy input texts. You see, language models charge us based on the number of tokens we use. We don't want to pay more just because our input has extra characters. It's not cool 😉
</br></br>
So, here's the plan: we'll create a special function to tidy up the spacing in our texts. Then, we'll use this function to make a chain. We'll feed in our messy text, and we want a nice, clean text as the result. Easy, right?

In [61]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}


Most importantly, when we start the chain, we don't include an "llm" as a parameter to it. Without this "llm," the chain isn't as strong as the one we looked at before. But, as we'll see soon, if we combine this chain with others, we can get really good outcomes.

In [63]:
clean_extra_spaces_chain = TransformChain(
    input_variables = ['text'],
    output_variables = ['output_text'],
    transform = transform_func
)

clean_extra_spaces_chain.run("This is an exampe     of incorrect spacing and new line. \n\n\n Let's see how chain works.")

"This is an exampe of incorrect spacing and new line. \n Let's see how chain works."

Awesome! Now, let's make things more fun.

Imagine we want to take some text, clean it up, and then rewrite it in a special way, like making it sound poetic or like a police report. Since the `TransformChain` doesn't handle styling, we need another tool for that, and that's where our `LLMChain` comes into play. We've talked about this chain before, and we can do some cool stuff with clever prompts, so let's give it a try!

First, we'll create the template for our prompt:

In [64]:
template = """Paraphrase this text:
{output_text}
In the style of a {style}.
Paraphrase:  """

prompt = PromptTemplate(
    input_variables = ['style', 'output_text'],
    template = template
)

In [65]:
style_paraphrase_chain = LLMChain(llm = llm, prompt = prompt, output_key = "final_output")

Awesome! See how the input text in the template is named 'output_text'? Any idea why?

We'll send what comes out of the `TransformChain` to the `LLMChain`!

Last step, we have to join them together to act as a single chain. To do that, we'll use `SequentialChain`, our third basic building block for chains.

Let's start by creating the prompt template:

In [67]:
sequential_chain = SequentialChain(
    chains = [clean_extra_spaces_chain, style_paraphrase_chain],
    input_variables = ['text', 'style'],
    output_variables=['final_output']
    )

In [68]:
input_text = """
Chains allow us to combine multiple


components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

In [77]:
count_tokens(sequential_chain, {"text" : input_text, "style" : "a singer"})

Spent a total of 178 tokens


'\nChains let us join together many pieces \nTo make one application that makes sense. \nFor instance, we can make a chain that takes user input, \nFormats it with a PromptTemplate, and then sends it to an LLM. \nWe can make more intricate chains by joining multiple chains, \nOr by combining chains with other components.'

#A note on langchain-hub
langchain-hub is a sister library to langchain, where all the chains, agents and prompts are serialized for us to use.

In [78]:
from langchain.chains import load_chain

Loading from the LangChain Hub is simple. Just find the chain you want in the repository and use `load_chain` with the right path. There are also "load_prompt" and `initialize_agent`," but we'll talk about those later. For now, let's focus on loading our `LLMMathChain` that we looked at earlier.

In [80]:
llm_math_chain = load_chain("lc://chains/llm-math/chain.json")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [82]:
# We can change the parameters as well

llm_math_chain.verbose

True

In [83]:
llm_math_chain.verbose = False

In [84]:
llm_math_chain.verbose

False